In [2]:
import pandas as pd
import numpy as np
import ast

In [42]:
df = pd.read_csv("naver_food_info.csv")
print(df.shape)


(7171, 7)


In [38]:
# 행이 동일한 중복된 행 찾기(크롤링 시간에 따라 영업시간 다름 이슈로 id 기준 중복 제거하기로 함)
# print(len(df[df.duplicated()]))
# print(df[df.duplicated()])

# 눈으로 확인하고 싶어서~
# duplicates = df[df.duplicated()]
# duplicates.to_csv("duplicates.csv", index=False)

# 중복 데이터 제거
# df = df.drop_duplicates()
# print(df.shape)


(6710, 7)


In [44]:
# 중복된 id를 가진 행들 찾기
duplicates = df[df.duplicated(subset=["id"], keep="first")]
# 개수 출력
print(len(duplicates))
# id 중복 행 제거
df = df.drop_duplicates(subset=['id'], keep='first')
print(df.shape)

575
(6596, 7)


In [45]:
# ID 범위
expected_ids = set(range(1, 6606))  # 1부터 6605까지의 ID

# 실제로 있는 ID
actual_ids = set(df['id'])

# 없는 ID 찾기
missing_ids = sorted(expected_ids - actual_ids)

print("누락된 ID:", missing_ids)

누락된 ID: [2992, 2993, 2994, 2995, 2996, 2997, 2998, 2999, 3000]


In [46]:
# 누락된 id 다시 크롤링해서 집어넣음
df.to_csv("naver_food_info_2.csv", index=False)

In [73]:
# 중복 제거를 위해 다시 읽기
df = pd.read_csv("naver_food_info_2.csv")
print(df.shape)

(6605, 7)


In [74]:
# 빈 문자열이나 None을 NaN으로 변경
df.replace({"": np.nan}, inplace=True)
# 결과 확인
print(df)

        id domain store_name category  rating                  address  \
0        1  naver        NaN      NaN     NaN                      NaN   
1        2  naver        NaN      NaN     NaN                      NaN   
2        3  naver        NaN      NaN     NaN                      NaN   
3        4  naver        NaN      NaN     NaN                      NaN   
4        5  naver        NaN      NaN     NaN                      NaN   
...    ...    ...        ...      ...     ...                      ...   
6600  6601  naver       고흥식당       한식     4.4           서울 종로구 종로34길 4   
6601  6602  naver        NaN      NaN     NaN                      NaN   
6602  6603  naver        NaN      NaN     NaN                      NaN   
6603  6604  naver        NaN      NaN     NaN                      NaN   
6604  6605  naver   콘크리트 팔레트   카페,디저트     NaN  서울 종로구 대학로 125 콘크리트 팔레트   

                                         business_hours  
0                                                   N

In [82]:
# 중복된 id를 가진 행들 찾기
# duplicates = df[df.duplicated(subset=["id"], keep="first")]
# 개수 출력 -> 0
# print(len(duplicates))

# 데이터 빈 값이 아니고, store_name 기준으로 중복된 데이터
# duplicates = df[df["store_name"].notna() & df.duplicated(subset=["store_name"], keep=False)]
# 개수 출력 -> 2710(.......?)
# print(len(duplicates))
# duplicates.to_csv("duplicates.csv", index=False)

# 이름은 같지만 주소는 다른 동명 가게들도 있음 확인 -> 이름과 주소가 같은 데이터 행 값 제거
# duplicates = df[df["store_name"].notna() & (df["store_name"] != "")].loc[df.duplicated(subset=["store_name", "address"], keep="first")]
# print(len(duplicates))
# 중복 행에서 id와 domain 컬럼을 제외한 나머지 컬럼을 NaN으로 설정
# columns_to_nan = [col for col in df.columns if col not in ["id", "domain"]]
# df.loc[duplicates.index, columns_to_nan] = np.nan
# store_name 컬럼에서 값이 있는 행의 개수 세기
# store_name_count = df[df["store_name"].notna() & (df["store_name"] != "")].shape[0]
# print(store_name_count)

# 이름, 카테고리, 별점 똑같은데 주소가 없어서 필터링 안된 거 찾기
# duplicates = df[df["store_name"].notna() & (df["store_name"] != "")].duplicated(subset=["store_name", "address"], keep=False)
duplicates = df[df["store_name"].notna() & (df["store_name"] != "") & df["address"].isna()].loc[df.duplicated(subset=["store_name", "category", "rating"], keep="first")]
print(duplicates)
# 중복 행에서 id와 domain 컬럼을 제외한 나머지 컬럼을 NaN으로 설정
columns_to_nan = [col for col in df.columns if col not in ["id", "domain"]]
df.loc[duplicates.index, columns_to_nan] = np.nan
# store_name 컬럼에서 값이 있는 행의 개수 세기
store_name_count = df[df["store_name"].notna() & (df["store_name"] != "")].shape[0]
print(store_name_count)

        id domain   store_name  category  rating address business_hours
1705  1706  naver  신선식탁 대학로 본점  다이어트,샐러드    4.54     NaN            NaN
1919  1920  naver  원조 순희네육회빈대떡     전,빈대떡    4.38     NaN            NaN
2091  2092  naver     종로 은행나무집   육류,고기요리    4.37     NaN            NaN
4051


In [109]:
# 컬럼명 변경
df = df.rename(columns={"store_name": "name"})
# 컬럼 추가
df['price_per_one'] = np.nan
df


,id,domain,name,category,rating,address,business_hours,price_per_one
0,1,naver,NaN,NaN,NaN,NaN,NaN,NaN
1,2,naver,NaN,NaN,NaN,NaN,NaN,NaN
2,3,naver,NaN,NaN,NaN,NaN,NaN,NaN
3,4,naver,NaN,NaN,NaN,NaN,NaN,NaN
4,5,naver,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6600,6601,naver,고흥식당,한식,4.4,서울 종로구 종로34길 4,NaN,NaN
6601,6602,naver,NaN,NaN,NaN,NaN,NaN,NaN
6602,6603,naver,NaN,NaN,NaN,NaN,NaN,NaN
6603,6604,naver,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
# 4051 데이터 저장저장!!
df.to_csv("naver_food_info_3.csv", index=False)

In [3]:
# 다시 읽기
df = pd.read_csv("naver_food_info_3.csv")
print(sum(df["name"].notna()))
print(sum(df["category"].notna()))

4051
4051


In [4]:
# 연관 없는 카테고리 삭제
# 카테고리 종류 확인
print(len(df["category"].unique()))
print(df["category"].unique())

200
[nan '바(BAR)' '생선회' '피자' '백숙,삼계탕' '분식' '카페' '브런치카페' '한식' '카페,디저트' '패션'
 '육류,고기요리' '국밥' '우동,소바' '소고기구이' '종합분식' '장어,먹장어요리' '일식당' '양식' '이탈리아음식'
 '한정식' '햄버거' '베이커리' '딤섬,중식만두' '치킨,닭강정' '순대,순댓국' '후렌치후라이' '이자카야' '요리주점'
 '맥주,호프' '서점' '포장마차' '아이스크림' '감자탕' '중식당' '일본식라면' '돈가스' '스파게티,파스타전문'
 '일식튀김,꼬치' '베트남음식' '두부요리' '전복요리' '돼지고기구이' '와인' '닭요리' '태국음식' '다이어트,샐러드'
 '갤러리,화랑' '술집' '프랑스음식' '칼국수,만두' '떡볶이' '카레' '곰탕,설렁탕' '브런치' '주류' '국수' '공방'
 '아시아음식' '퓨전음식' '양꼬치' '오징어요리' '바나프레소' '독일음식' '해물,생선요리' '과일,주스전문점'
 '전통,민속주점' '보드카페' '북카페' '라면' '샌드위치' '차' '패밀리레스토랑' '멕시코,남미음식' '만두'
 '테이크아웃커피' '매운탕,해물탕' '인테리어소품' '오뎅,꼬치' '도시락,컵밥' '이북음식' '주류제조' '마라탕'
 '곱창,막창,양' '해장국' '추어탕' '덮밥' '인도음식' '죽' '장소대여' '김밥' '초밥,롤' '꽃집,꽃배달' '농수산물'
 '한식뷔페' '푸드코트' '호텔' '샤브샤브' '토스트' '테마카페' '한방카페' '찌개,전골' '냉면' '공연장' '중개업'
 '도서,음반,문구' '섬유,의류' '족발,보쌈' '복합문화공간' '72420' '도시락,조리식품제조' '라이브카페' '생선구이'
 '자동판매기' '조개요리' '막국수' '낙지요리' '슈퍼,마트' '닭갈비' '사주카페' '모텔' '비빔밥' '전통숙소' '전시관'
 '차,커피' '전,빈대떡' '키즈카페,실내놀이터' '떡카페' '정육식당' '와플' '건강기능보조식품' '블루보틀' '미술관'
 '터키음식' '

In [5]:

# 유지 리스트(음식점, 카페, 술집) = ['바(BAR)', '생선회', '피자', '백숙,삼계탕', '분식', '카페', 
# '브런치카페', '한식', '카페,디저트', '육류,고기요리', '국밥', '우동,소바', '소고기구이', '종합분식', 
# '장어,먹장어요리', '일식당', '양식', '이탈리아음식', '한정식', '햄버거', '베이커리', '딤섬,중식만두',
# '치킨,닭강정', '순대,순댓국', '후렌치후라이', '이자카야', '요리주점', '맥주,호프', '포장마차', 
# '아이스크림', '감자탕', '중식당', '일본식라면', '돈가스', '스파게티,파스타전문', '일식튀김,꼬치', 
# '베트남음식', '두부요리', '전복요리', '돼지고기구이', '와인', '닭요리', '태국음식', '다이어트,샐러드', 
# '술집', '프랑스음식', '칼국수,만두', '떡볶이', '카레', '곰탕,설렁탕', '브런치', '국수', '아시아음식', 
# '퓨전음식', '양꼬치', '오징어요리', '바나프레소', '독일음식', '해물,생선요리', '과일,주스전문점', 
# '전통,민속주점', '라면', '샌드위치', '차', '패밀리레스토랑', '멕시코,남미음식', '만두', '테이크아웃커피',
# '매운탕,해물탕', '오뎅,꼬치', '도시락,컵밥', '이북음식', '마라탕', '곱창,막창,양', '해장국', '추어탕', 
# '덮밥', '인도음식', '죽', '김밥', '초밥,롤', '한식뷔페', '샤브샤브', '토스트', '한방카페', 
# '찌개,전골', '냉면', '족발,보쌈', '72420', '라이브카페', '생선구이', '조개요리', '막국수', '낙지요리', 
# '슈퍼,마트', '닭갈비', '비빔밥', '차,커피', '전,빈대떡', '떡카페', '정육식당', '와플', '백반,가정식', 
# '주꾸미요리', '사철,영양탕', '블루보틀', '터키음식', '호떡', '스테이크,립', '아귀찜,해물찜', '양갈비', 
# '찜닭', '신의주부대찌개', '핫도그', '뷔페', '사찰음식', '닭볶음탕', '복어요리', '닭발', '빙수', 
# '초콜릿전문점', '스페인음식', '케이크전문', '향토음식', '보리밥', '유흥주점', '굴요리', '쌈밥', 
# '오리요리', '소고기', '룸카페', '다방', '야식', '갈비탕', '채식,샐러드뷔페', '바닷가재요리', 
# '음식점', '도넛']

# 삭제할 카테고리 리스트
exclude_categories = ['패션', '서점', '갤러리,화랑', '주류', '공방', '보드카페', '북카페', 
       '인테리어소품', '주류제조', '장소대여', '꽃집,꽃배달', '농수산물', '푸드코트', '호텔', '공연장', 
       '중개업', '도서,음반,문구', '섬유,의류', '복합문화공간', '도시락,조리식품제조', '자동판매기', 
       '사주카페', '모텔', '전통숙소', '전시관', '키즈카페,실내놀이터', '건강기능보조식품', '미술관', 
       '반찬가게', '기업', '여성의류', '종합패션', '예식장', '볼링장', '문화원', '기타숙박업', 
       '전시,행사대행', '만화방', '한복', '미술,공예품', '3급', '교육원,교육센터', '패션잡화', 
       '양말', '독립서점', '여행사', '갤러리카페', '식료품', '요리교육', '가구,인테리어', '보세의류', 
       '운세,사주', '화장품,향수', '밀키트', '인테리어디자인', '기업,빌딩', '스터디카페', '도자기',
       '두부제조', '한복대여', '테마카페']
print(len(exclude_categories))
# 특정 카테고리를 포함하는 행을 찾고, 해당 행의 나머지 컬럼을 NaN으로 설정 -> 4051-132 = 3919
print(len(df.loc[df['category'].isin(exclude_categories)]))
df.loc[df['category'].isin(exclude_categories), ['name', 'category', 'rating', 'address', 'business_hours', 'price_per_one']] = np.nan
print(sum(df["name"].notna()))

61
132
3919


In [126]:
# 남은 카테고리 대분류 지정
# main_category: 카페, 음식점, 주점

# category 앞에 main_category 컬럼 추가
df.insert(df.columns.get_loc('category'), 'main_category', np.nan)
print(df.head())

   id domain name  main_category category  rating address business_hours  \
0   1  naver  NaN            NaN      NaN     NaN     NaN            NaN   
1   2  naver  NaN            NaN      NaN     NaN     NaN            NaN   
2   3  naver  NaN            NaN      NaN     NaN     NaN            NaN   
3   4  naver  NaN            NaN      NaN     NaN     NaN            NaN   
4   5  naver  NaN            NaN      NaN     NaN     NaN            NaN   

   price_per_one  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  


In [7]:
# list 구성
restaurant = ['생선회', '피자', '백숙,삼계탕', '분식', '브런치카페', '한식', '육류,고기요리', 
              '국밥', '우동,소바', '소고기구이', '종합분식', '장어,먹장어요리', '일식당', '양식', 
              '이탈리아음식', '한정식', '햄버거', '베이커리', '딤섬,중식만두', '치킨,닭강정', 
              '순대,순댓국', '후렌치후라이', '감자탕', '중식당', '일본식라면', '돈가스', 
              '스파게티,파스타전문', '일식튀김,꼬치', '베트남음식', '두부요리', '전복요리', 
              '돼지고기구이', '닭요리', '태국음식', '다이어트,샐러드', '프랑스음식', '칼국수,만두', 
              '떡볶이', '카레', '곰탕,설렁탕', '브런치', '국수', '아시아음식', '퓨전음식', 
              '양꼬치', '오징어요리', '독일음식', '해물,생선요리', '전통,민속주점', '라면', 
              '샌드위치', '패밀리레스토랑', '멕시코,남미음식', '만두', '매운탕,해물탕', '오뎅,꼬치', 
              '도시락,컵밥', '이북음식', '마라탕', '곱창,막창,양', '해장국', '추어탕', '덮밥', 
              '인도음식', '죽', '김밥', '초밥,롤', '한식뷔페', '샤브샤브', '토스트', '찌개,전골', 
              '냉면', '족발,보쌈', '72420', '생선구이', '조개요리', '막국수', '낙지요리', 
              '닭갈비', '비빔밥', '전,빈대떡', '정육식당', '백반,가정식', '주꾸미요리', 
              '사철,영양탕', '터키음식', '스테이크,립', '아귀찜,해물찜', '양갈비', '찜닭', 
              '신의주부대찌개', '핫도그', '뷔페', '사찰음식', '닭볶음탕', '복어요리', '닭발', 
              '스페인음식', '향토음식', '보리밥', '굴요리', '쌈밥', '오리요리', '소고기', '야식', 
              '갈비탕', '채식,샐러드뷔페', '바닷가재요리', '음식점']
cafe = ['카페', '카페,디저트', '바나프레소', '차', '블루보틀', '빙수', '초콜릿전문점', '룸카페', 
        '케이크전문', '아이스크림', '과일,주스전문점', '테이크아웃커피', '한방카페', '차,커피', 
        '떡카페', '와플', '호떡', '다방', '도넛']
pub = ['바(BAR)', '이자카야', '요리주점', '맥주,호프', '포장마차', '와인', '술집', '라이브카페', 
       '슈퍼,마트', '유흥주점']
print(len(restaurant))
print(len(cafe))
print(len(pub))

109
19
10


In [129]:
def assign_main_category(category):
    if pd.isna(category):  # category가 NaN일 경우
        return None  # None을 반환해서 main_category도 NaN으로 설정
    elif category in restaurant:
        return "음식점"
    elif category in cafe:
        return "카페"
    elif category in pub:
        return "주점"
    else:
        return "기타"

# main_category 값 설정
df["main_category"] = df["category"].apply(assign_main_category)

In [134]:
print(df[df["main_category"] == "기타"])
print(df["main_category"].value_counts())
print(df["main_category"].notna().sum())

Empty DataFrame
Columns: [id, domain, name, main_category, category, rating, address, business_hours, price_per_one]
Index: []
main_category
음식점    2801
카페      581
주점      537
Name: count, dtype: int64
3919


In [136]:
# 3919 데이터 저장저장!! 완성 및 이름 변경 -> row_data/naver_store_info.csv
df.to_csv("naver_food_info_4.csv", index=False)

In [157]:
# naver_food_info_4.csv id 기반으로 리뷰데이터 매칭
info = pd.read_csv("naver_food_info_4.csv")
print(info.shape)
print(sum(info["name"].notna()))
# print(sum(info["category"].notna()))
# 리뷰 데이터 읽기
reviews = pd.read_csv("naver_review.csv")
print(reviews.shape)
print(sum(reviews["review"].notna()))
print(len(reviews["id"].unique()))

(6605, 9)
3919
(98809, 2)
95943
3735


In [158]:
# 리뷰 데이터 확인
# print(reviews)
# id 컬럼을 정수로 변환
reviews["id"] = reviews["id"].astype(int)
# 컬럼명 변경
reviews.rename(columns={"review": "reviews"}, inplace=True)
# print(reviews)
# reviews 컬럼이 NaN인 행 삭제
reviews = reviews.dropna(subset=["reviews"])
print(reviews.shape)
print(reviews)

(95943, 2)
         id                                            reviews
0        18  혜화동 새로 오픈한 신상 피자집! 대학로 cgv 영화관람하고 왔어요. 시그니처 바질...
1        18  JMT !!!!!!!!!! 노모어피자 하도 유행해서 먹어봤는데 왜 먹는지 알겠더라고...
2        18  대학로에. 노모어피자가. 생기다니!!!!!! 여기 사장님이 노모어피자 초창기 운영하...
3        18  피자가 빨리 나오고 맛있어요! 매장 안에 화장실 있어서 편리합니당. 아이랑 갔는데 ...
4        18  새로 생겨서 가봤는데 진짜 넘 맛도링 ~~~~\n피자는 물론이고 바질뇨끼 진짜 맛있어요!
...     ...                                                ...
98804  6605  오늘은 까눌레가 늦게 나오네용 ~ 🥲\n그래서 저번이랑 똑같이 주문했다는 ~\n커피...
98805  6605  오~~~랜만에 대학로 ~ 👀🤟🏻✨\n일행 기다려야 해서 잠시 카페 들어 왔는데\n아...
98806  6605       밤라떼 달지 않고 맛있어요! 까눌레도 겉바속쫀득 완벽해서 커피랑 꿀조합입니다:)
98807  6605  주말 낮에 자리 여유많았어요\n2층도 있었어요 통유리라 낮에 유리쪽은 좀 덥긴했는데...
98808  6605  혜화 오다가며 봤던 곳이었는데, 전체가 카페였다니ㅋㅋㄱ 상상도 못했네요!\n콘크리트...

[95943 rows x 2 columns]


In [159]:
# reviews & info -> id 기준으로 병합
mg_reviews = reviews.merge(info[["id", "domain", "name"]], on="id", how="left")
# 컬럼 순서 조정 (id -> domain -> name -> reviews)
mg_reviews = mg_reviews[["id", "domain", "name", "reviews"]]
print(mg_reviews)

         id domain        name  \
0        18  naver  노모어피자 대학로점   
1        18  naver  노모어피자 대학로점   
2        18  naver  노모어피자 대학로점   
3        18  naver  노모어피자 대학로점   
4        18  naver  노모어피자 대학로점   
...     ...    ...         ...   
95938  6605  naver    콘크리트 팔레트   
95939  6605  naver    콘크리트 팔레트   
95940  6605  naver    콘크리트 팔레트   
95941  6605  naver    콘크리트 팔레트   
95942  6605  naver    콘크리트 팔레트   

                                                 reviews  
0      혜화동 새로 오픈한 신상 피자집! 대학로 cgv 영화관람하고 왔어요. 시그니처 바질...  
1      JMT !!!!!!!!!! 노모어피자 하도 유행해서 먹어봤는데 왜 먹는지 알겠더라고...  
2      대학로에. 노모어피자가. 생기다니!!!!!! 여기 사장님이 노모어피자 초창기 운영하...  
3      피자가 빨리 나오고 맛있어요! 매장 안에 화장실 있어서 편리합니당. 아이랑 갔는데 ...  
4      새로 생겨서 가봤는데 진짜 넘 맛도링 ~~~~\n피자는 물론이고 바질뇨끼 진짜 맛있어요!  
...                                                  ...  
95938  오늘은 까눌레가 늦게 나오네용 ~ 🥲\n그래서 저번이랑 똑같이 주문했다는 ~\n커피...  
95939  오~~~랜만에 대학로 ~ 👀🤟🏻✨\n일행 기다려야 해서 잠시 카페 들어 왔는데\n아...  
95940       밤라떼 달지 않고 맛있어요! 까눌레도 겉바속쫀득 완벽해서 커피랑 꿀조합입니다:)  
9

In [160]:
# info에 name 컬럼이 존재하는 경우에만 reviews 데이터프레임의 행을 유지하고, 그렇지 않으면 삭제
# info에 해당 id의 name 값이 있는지 확인
valid_ids = info.loc[info["name"].notna(), "id"]
# mg_reviews에서 id가 valid_ids에 포함된 행만 유지
filtered_reviews = mg_reviews[mg_reviews["id"].isin(valid_ids)]
print(filtered_reviews)

         id domain        name  \
0        18  naver  노모어피자 대학로점   
1        18  naver  노모어피자 대학로점   
2        18  naver  노모어피자 대학로점   
3        18  naver  노모어피자 대학로점   
4        18  naver  노모어피자 대학로점   
...     ...    ...         ...   
95938  6605  naver    콘크리트 팔레트   
95939  6605  naver    콘크리트 팔레트   
95940  6605  naver    콘크리트 팔레트   
95941  6605  naver    콘크리트 팔레트   
95942  6605  naver    콘크리트 팔레트   

                                                 reviews  
0      혜화동 새로 오픈한 신상 피자집! 대학로 cgv 영화관람하고 왔어요. 시그니처 바질...  
1      JMT !!!!!!!!!! 노모어피자 하도 유행해서 먹어봤는데 왜 먹는지 알겠더라고...  
2      대학로에. 노모어피자가. 생기다니!!!!!! 여기 사장님이 노모어피자 초창기 운영하...  
3      피자가 빨리 나오고 맛있어요! 매장 안에 화장실 있어서 편리합니당. 아이랑 갔는데 ...  
4      새로 생겨서 가봤는데 진짜 넘 맛도링 ~~~~\n피자는 물론이고 바질뇨끼 진짜 맛있어요!  
...                                                  ...  
95938  오늘은 까눌레가 늦게 나오네용 ~ 🥲\n그래서 저번이랑 똑같이 주문했다는 ~\n커피...  
95939  오~~~랜만에 대학로 ~ 👀🤟🏻✨\n일행 기다려야 해서 잠시 카페 들어 왔는데\n아...  
95940       밤라떼 달지 않고 맛있어요! 까눌레도 겉바속쫀득 완벽해서 커피랑 꿀조합입니다:)  
9

In [161]:
# 91598 데이터 저장저장!! 완성 및 이름 변경 -> row_data/naver_store_reviews.csv
filtered_reviews.to_csv("naver_store_reviews.csv", index=False)

In [4]:
# 확인
reviews = pd.read_csv("naver_store_reviews.csv")
print(reviews.shape)
print(sum(reviews["reviews"].notna()))
print(len(reviews["id"].unique()))

(91598, 4)
91598
3559
